In [2]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark com a configuração do Delta Lake
spark = SparkSession.builder \
    .appName("DeltaLake Example") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Verificar se a sessão Spark foi criada com sucesso
print(spark.version)

3.5.0


In [12]:
df_test = spark.read.csv("./data/raw/temas_ambientais.csv", sep=";", header=True)
df_test.show()

+---+--------------------+-----------+--------------+--------------------+-----------------+--------------------+------------+----------------------------------+---------------------------+---------------------------+---------------------+----------------------+-----------------+-----------------+--------------------+--------------------------------+----------------------+----------------------------+-----------------+---------------+-----------------+---------------------------+----------------------------------------+----------------+-----------------------+
| uf|           municipio|codigo_ibge|area_do_imovel|        registro_car|situacao_cadastro|   condicao_cadastro|area_liquida|area_remanescente_vegetacao_nativa|area_reserva_legal_proposta|area_preservacao_permanente|area_nao_classificada|solicitacao_adesao_pra|         latitude|        longitude|      data_inscricao|data_alteracao_condicao_cadastro|area_rural_consolidada|area_servidao_administrativa|tipo_imovel_rural|modulos_fis

In [ ]:
int_columns = ["codigo_ibge"]
float_columns = ["area_do_imovel", "area_liquida", "area_remanescente_vegetacao_nativa", 
                 "area_reserva_legal_proposta", "area_preservacao_permanente", "area_nao_classificada",
                 "latitude", "longitude", "area_rural_consolidada", "area_servidao_administrativa",
                 "modulos_fiscais", "area_uso_restrito", "area_reserva_legal_averbada",
                 "area_reserva_legal_aprovada_nao_averbada", "area_pousio"]
bool_columns = ["solicitacao_adesao_pra"]
date_columns = ["data_inscricao", "data_alteracao_condicao_cadastro", "data_ultima_retificacao"]
text_column = [x for x in df_test.columns if x not in int_columns+float_columns+bool_columns+date_columns]

['uf', 'municipio', 'registro_car', 'situacao_cadastro', 'condicao_cadastro', 'tipo_imovel_rural']


26

In [18]:
df_test.printSchema()

root
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- codigo_ibge: string (nullable = true)
 |-- area_do_imovel: string (nullable = true)
 |-- registro_car: string (nullable = true)
 |-- situacao_cadastro: string (nullable = true)
 |-- condicao_cadastro: string (nullable = true)
 |-- area_liquida: string (nullable = true)
 |-- area_remanescente_vegetacao_nativa: string (nullable = true)
 |-- area_reserva_legal_proposta: string (nullable = true)
 |-- area_preservacao_permanente: string (nullable = true)
 |-- area_nao_classificada: string (nullable = true)
 |-- solicitacao_adesao_pra: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- data_inscricao: string (nullable = true)
 |-- data_alteracao_condicao_cadastro: string (nullable = true)
 |-- area_rural_consolidada: string (nullable = true)
 |-- area_servidao_administrativa: string (nullable = true)
 |-- tipo_imovel_rural: string (nullable = true)
